## Deel 1: omloopplanning en dienstregeling controleren

### Functies en inladen

In [50]:
# Importeren van benodigde libraries
import pandas as pd
from IPython.display import display
import warnings

# Waarschuwingen negeren
warnings.filterwarnings('ignore')

# Functie om data in te laden
def load_data():
    # Lees de Excel-bestanden in DataFrames
    df_omloopplanning = pd.read_excel('omloopplanning.xlsx', engine='openpyxl')
    df_dienstregeling = pd.read_excel('Connexxion data - 2024-2025.xlsx', engine='openpyxl')
    return df_omloopplanning, df_dienstregeling

def check_omloopplanning(omloop_df, dienst_df):
    # Voeg een nieuwe kolom toe om de correctheid te markeren
    omloop_df['correct'] = False

    # Filter alleen op dienstritten in omloopplanning
    dienst_ritten_omloop = omloop_df[omloop_df['activiteit'] == 'dienst rit']
    
    for idx, row in dienst_ritten_omloop.iterrows():
        # Filter voor de overeenkomstige rijen in de dienstregeling
        dienst_rows = dienst_df[
            (dienst_df['startlocatie'] == row['startlocatie']) &
            (dienst_df['eindlocatie'] == row['eindlocatie']) &
            (dienst_df['buslijn'] == row['buslijn'])
        ]
        
        for _, dienst_row in dienst_rows.iterrows():
            # Maak een vertrektijd datetime object
            if pd.isna(row['starttijd']):
                print(f"Skipping row {idx} because of NaT (Not a Time)")
                continue
            
            vertrektijd = pd.to_datetime(f"{row['starttijd'].date()} {dienst_row['vertrektijd'].strip()}")
            
            # Controleer of de starttijd overeenkomt met de dienstregeling
            if vertrektijd == row['starttijd']:
                omloop_df.at[idx, 'correct'] = True
                break

    # Controleer of alle ritten in de dienstregeling aanwezig zijn in de omloopplanning
    dienst_df['found_in_omloop'] = False

    for idx, row in dienst_df.iterrows():
        try:
            omloop_rows = omloop_df[
                (omloop_df['startlocatie'] == row['startlocatie']) &
                (omloop_df['eindlocatie'] == row['eindlocatie']) &
                (omloop_df['buslijn'] == row['buslijn']) &
                (omloop_df['starttijd'].dt.time == pd.to_datetime(f"{row['vertrektijd']}").time()) &
                (omloop_df['activiteit'] == 'dienst rit')
            ]
        except ValueError as e:
            print(f"Error in row {idx}: {str(e)}")
            continue

        if not omloop_rows.empty:
            dienst_df.at[idx, 'found_in_omloop'] = True

    return omloop_df, dienst_df

In [28]:
# Inladen van data
df_omloopplanning, df_dienstregeling = load_data()

# Converteer tijd kolommen naar datetime objecten voor makkelijke bewerking
df_omloopplanning['starttijd'] = pd.to_datetime(df_omloopplanning['starttijd datum'], errors='coerce')
df_omloopplanning['eindtijd'] = pd.to_datetime(df_omloopplanning['eindtijd datum'], errors='coerce')

# Voer de controle uit
df_omloopplanning, df_dienstregeling = check_omloopplanning(df_omloopplanning, df_dienstregeling)

# Controleer de eerste paar rijen van de uitkomst om te zien of de correctheid is bijgewerkt
display(df_omloopplanning.head())
display(df_dienstregeling.head())

Error in row 326: NaTType does not support time


,Unnamed: 0,startlocatie,eindlocatie,starttijd,eindtijd,activiteit,buslijn,energieverbruik,starttijd datum,eindtijd datum,omloop nummer,Unnamed: 11,correct
0,0,ehvgar,ehvbst,2024-08-29 05:03:00,2024-08-29 05:07:00,materiaal rit,NaN,1.9800,2024-08-29 05:03:00,2024-08-29 05:07:00,1,47.5200,False
1,1,ehvbst,ehvapt,2024-08-29 05:07:00,2024-08-29 05:31:00,dienst rit,401.0,10.8036,2024-08-29 05:07:00,2024-08-29 05:31:00,1,259.2864,True
2,2,ehvapt,ehvapt,2024-08-29 05:31:00,2024-08-29 06:04:00,idle,NaN,0.0100,2024-08-29 05:31:00,2024-08-29 06:04:00,1,0.2400,False
3,3,ehvapt,ehvbst,2024-08-29 06:04:00,2024-08-29 06:29:00,dienst rit,401.0,10.8600,2024-08-29 06:04:00,2024-08-29 06:29:00,1,NaN,True
4,4,ehvbst,ehvbst,2024-08-29 06:29:00,2024-08-29 06:31:00,idle,NaN,0.0100,2024-08-29 06:29:00,2024-08-29 06:31:00,1,NaN,False


,startlocatie,vertrektijd,eindlocatie,buslijn,found_in_omloop
0,ehvapt,06:04,ehvbst,401.0,True
1,ehvapt,06:34,ehvbst,401.0,True
2,ehvapt,06:56,ehvbst,401.0,True
3,ehvapt,07:11,ehvbst,401.0,True
4,ehvapt,07:26,ehvbst,401.0,True


### Daadwerkelijke controle met dienstregeling

In [29]:
# Deel 1: Resultaten van omloopplanning controleren
print('Resultaten van Omloopplanning Controle')
filtered_df = df_omloopplanning[df_omloopplanning['activiteit'] == 'dienst rit']
false_count = filtered_df['correct'].value_counts().get(False, 0)
print(f"Aantal onjuiste dienstritten in omloopplanning: {false_count}")
if false_count > 0:
    false_rows = filtered_df[filtered_df['correct'] == False]
    print("Onjuiste dienstritten in omloopplanning:")
    display(false_rows[['startlocatie', 'eindlocatie', 'starttijd', 'eindtijd', 'buslijn', 'correct']])

Resultaten van Omloopplanning Controle
Aantal onjuiste dienstritten in omloopplanning: 1
Onjuiste dienstritten in omloopplanning:


,startlocatie,eindlocatie,starttijd,eindtijd,buslijn,correct
575,ehvbst,ehvapt,2024-08-29 19:37:00,2024-08-29 20:01:00,400.0,False


In [30]:
# Deel 2: Resultaten van dienstregeling controleren
print('\nResultaten van Dienstregeling Controle')
not_found_count = df_dienstregeling['found_in_omloop'].value_counts().get(False, 0)
print(f"Aantal dienstritten in dienstregeling die niet in omloopplanning zijn gevonden: {not_found_count}")
if not_found_count > 0:
    not_found_rows = df_dienstregeling[df_dienstregeling['found_in_omloop'] == False]
    print("\nDienstritten in dienstregeling die niet in de omloopplanning zijn gevonden:")
    display(not_found_rows[['startlocatie', 'eindlocatie', 'vertrektijd', 'buslijn']])


Resultaten van Dienstregeling Controle
Aantal dienstritten in dienstregeling die niet in omloopplanning zijn gevonden: 2

Dienstritten in dienstregeling die niet in de omloopplanning zijn gevonden:


,startlocatie,eindlocatie,vertrektijd,buslijn
326,NaN,NaN,NaN,NaN
327,ehvbst,foute test data,19:45,395.0


## Deel 2: Totaal energie verbruik bepalen

In [31]:
import pandas as pd
# Eerst ervoor zorgen dat de kolommen 'starttijd' en 'eindtijd' in datetime geformatteerd zijn
df_omloopplanning['starttijd'] = pd.to_datetime(df_omloopplanning['starttijd'])
df_omloopplanning['eindtijd'] = pd.to_datetime(df_omloopplanning['eindtijd'])

# Nieuwe kolom 'duur' toevoegen die de verschil tussen eindtijd en starttijd aangeeft
df_omloopplanning['duur'] = df_omloopplanning['eindtijd'] - df_omloopplanning['starttijd']

# Als je de duur in minuten wilt omzetten
df_omloopplanning['duur_minuten'] = df_omloopplanning['duur'].dt.total_seconds() / 60

# Resultaat bekijken
# display(df_omloopplanning[['starttijd', 'eindtijd', 'duur', 'duur_minuten','energieverbruik']])

In [32]:
import pandas as pd

# Eerst ervoor zorgen dat de kolommen 'starttijd' en 'eindtijd' in datetime geformatteerd zijn
df_omloopplanning['starttijd'] = pd.to_datetime(df_omloopplanning['starttijd'])
df_omloopplanning['eindtijd'] = pd.to_datetime(df_omloopplanning['eindtijd'])

# Nieuwe kolom 'duur' toevoegen die het verschil tussen eindtijd en starttijd aangeeft
df_omloopplanning['duur'] = df_omloopplanning['eindtijd'] - df_omloopplanning['starttijd']

# De kolom 'duur' converteren naar minuten
df_omloopplanning['duur_minuten'] = df_omloopplanning['duur'].dt.total_seconds() / 60

# De kolom 'duur_minuten' converteren naar uren
df_omloopplanning['duur_uren'] = df_omloopplanning['duur_minuten'] / 60

# energieverbruik kolom: de eenheid is kWh --> dus NIET kWh per gereden kilometer

# Totale gebruikte kilowatturen (kWh) berekenen en opslaan in een nieuwe kolom 'totale_kWh'
df_omloopplanning['gebruikt_kW'] = df_omloopplanning['duur_uren'] * df_omloopplanning['energieverbruik']

# Resultaat bekijken
display(df_omloopplanning[['starttijd', 'eindtijd', 'duur_uren', 'energieverbruik', 'gebruikt_kW','omloop nummer']])

,starttijd,eindtijd,duur_uren,energieverbruik,gebruikt_kW,omloop nummer
0,2024-08-29 05:03:00,2024-08-29 05:07:00,0.066667,1.9800,0.132000,1
1,2024-08-29 05:07:00,2024-08-29 05:31:00,0.400000,10.8036,4.321440,1
2,2024-08-29 05:31:00,2024-08-29 06:04:00,0.550000,0.0100,0.005500,1
3,2024-08-29 06:04:00,2024-08-29 06:29:00,0.416667,10.8600,4.525000,1
4,2024-08-29 06:29:00,2024-08-29 06:31:00,0.033333,0.0100,0.000333,1
...,...,...,...,...,...,...
715,2024-08-29 17:06:00,2024-08-29 17:30:00,0.400000,12.8496,5.139840,19
716,2024-08-29 17:30:00,2024-08-29 17:50:00,0.333333,10.8000,3.600000,19
717,2024-08-29 14:02:00,2024-08-29 14:06:00,0.066667,1.9800,0.132000,20
718,2024-08-29 14:06:00,2024-08-29 14:30:00,0.400000,12.8496,5.139840,20


In [33]:
max_batt_capa = 300 # kW
SOC_start = 0.9 # factor
SOC_min = 0.1 # factor
batterijslijtage = 0.85 # Afhankelijk van de leeftijd van de bus is dat zo’n 85%-95% van de maximale capaciteit 
SOH =  max_batt_capa * batterijslijtage # De SOH is de maximale capaciteit van een specifieke bus

SOC_ochtend = SOH * SOC_start # De SOC geeft aan hoeveel procent de bus nog geladen is. 100% is daarbij gelijk aan de SOH van de bus.
SOC_minimum = SOH * SOC_min # De veiligheidsmarge van 10% heeft ook betrekking op de SOH

# DataFrame maken
data = {
    'Parameter': ['Max Batterij Capaciteit', 'SOC Start', 'SOC Minimum', 'Batterij Slijtage', 'SOH', 'SOC Ochtend', 'SOC Minimum'],
    'Waarde': [max_batt_capa, SOC_start, SOC_min, batterijslijtage, SOH, SOC_ochtend, SOC_minimum],
    'Eenheid': ['kW', 'Factor', 'Factor', 'Factor', 'kW (MBC * BS)', 'kW (SOH * SOC S)', 'kW (SOH * SOC M)']
}

df = pd.DataFrame(data)

# Resultaat weergeven
display(df)

,Parameter,Waarde,Eenheid
0,Max Batterij Capaciteit,300.00,kW
1,SOC Start,0.90,Factor
2,SOC Minimum,0.10,Factor
3,Batterij Slijtage,0.85,Factor
4,SOH,255.00,kW (MBC * BS)
5,SOC Ochtend,229.50,kW (SOH * SOC S)
6,SOC Minimum,25.50,kW (SOH * SOC M)


In [34]:
# Initialiseren van de kolommen voor SOC
df_omloopplanning['SOC_beginrit'] = 0.0
df_omloopplanning['SOC_eindrit'] = 0.0

# Berekenen van SOC_beginrit en SOC_eindrit per rit
huidige_omloop = None
huidige_SOC = SOC_ochtend

for index, row in df_omloopplanning.iterrows():
    if row['omloop nummer'] != huidige_omloop:
        huidige_omloop = row['omloop nummer']
        huidige_SOC = SOC_ochtend
    
    # SOC_beginrit instellen
    df_omloopplanning.at[index, 'SOC_beginrit'] = huidige_SOC
    
    # SOC_eindrit berekenen
    huidige_SOC -= row['gebruikt_kW']
    df_omloopplanning.at[index, 'SOC_eindrit'] = huidige_SOC

    # Controle of nodig is
    if huidige_SOC < SOC_minimum:
        print(f"Waarschuwing: SOC onder de minimum veiligheidsmarge voor omloop nummer {huidige_omloop} bij index {index}.")

# Toevoegen van nieuwe kolom die aangeeft of SOC_eindrit boven SOC_minimum is
df_omloopplanning['SOC_above_min'] = df_omloopplanning['SOC_eindrit'] > SOC_minimum

# Resultaat bekijken
display(df_omloopplanning[['omloop nummer', 'starttijd', 'eindtijd', 'duur_uren', 'energieverbruik', 'gebruikt_kW', 'SOC_beginrit', 'SOC_eindrit', 'SOC_above_min']])

,omloop nummer,starttijd,eindtijd,duur_uren,energieverbruik,gebruikt_kW,SOC_beginrit,SOC_eindrit,SOC_above_min
0,1,2024-08-29 05:03:00,2024-08-29 05:07:00,0.066667,1.9800,0.132000,229.50000,229.368000,True
1,1,2024-08-29 05:07:00,2024-08-29 05:31:00,0.400000,10.8036,4.321440,229.36800,225.046560,True
2,1,2024-08-29 05:31:00,2024-08-29 06:04:00,0.550000,0.0100,0.005500,225.04656,225.041060,True
3,1,2024-08-29 06:04:00,2024-08-29 06:29:00,0.416667,10.8600,4.525000,225.04106,220.516060,True
4,1,2024-08-29 06:29:00,2024-08-29 06:31:00,0.033333,0.0100,0.000333,220.51606,220.515727,True
...,...,...,...,...,...,...,...,...,...
715,19,2024-08-29 17:06:00,2024-08-29 17:30:00,0.400000,12.8496,5.139840,209.59696,204.457120,True
716,19,2024-08-29 17:30:00,2024-08-29 17:50:00,0.333333,10.8000,3.600000,204.45712,200.857120,True
717,20,2024-08-29 14:02:00,2024-08-29 14:06:00,0.066667,1.9800,0.132000,229.50000,229.368000,True
718,20,2024-08-29 14:06:00,2024-08-29 14:30:00,0.400000,12.8496,5.139840,229.36800,224.228160,True


In [35]:
# Filteren van de regels waarbij SOC_above_min False is
filtered_df = df_omloopplanning[df_omloopplanning['SOC_above_min'] == False]
filtered_df

,Unnamed: 0,startlocatie,eindlocatie,starttijd,eindtijd,activiteit,buslijn,energieverbruik,starttijd datum,eindtijd datum,omloop nummer,Unnamed: 11,correct,duur,duur_minuten,duur_uren,gebruikt_kW,SOC_beginrit,SOC_eindrit,SOC_above_min


In [36]:
# Berekenen van de laagste SOC_eindrit per omloopnummer
min_SOC_per_omloopnummer = df_omloopplanning.groupby('omloop nummer')['SOC_eindrit'].min().reset_index()
min_SOC_per_omloopnummer.columns = ['omloop nummer', 'min_SOC_eindrit']

# Resultaat bekijken
display(min_SOC_per_omloopnummer)

,omloop nummer,min_SOC_eindrit
0,1,134.647833
1,2,133.765393
2,3,132.551827
3,4,137.335213
4,5,137.267507
5,6,143.935660
6,7,140.897047
7,8,132.816507
8,9,137.747747
9,10,136.862140


## Oplaadtijd controleren

In [37]:
# Haal de unieke waarden uit de kolom 'activiteit'
unieke_waarden_activiteit = df_omloopplanning['activiteit'].unique()

# Print de unieke waarden
print("Unieke waarden in de kolom 'activiteit':")
print(unieke_waarden_activiteit)

Unieke waarden in de kolom 'activiteit':
['materiaal rit' 'dienst rit' 'idle' 'opladen']


In [52]:
# Filter de rijen waarbij 'activiteit' gelijk is aan 'opladen'
opladen_df = df_omloopplanning[df_omloopplanning['activiteit'] == 'opladen']

# Controleren of er rijen met 'opladen' zijn gevonden
if opladen_df.empty:
    print("Geen rijen gevonden waarbij de 'activiteit' gelijk is aan 'opladen'.")
else:
    # Voeg een nieuwe kolom toe die aangeeft of de 'duur_minuten' groter is dan 15
    opladen_df['lang_genoeg_opgeladen'] = opladen_df['duur_minuten'] > 15
    
    print("Rijen waarbij de 'activiteit' gelijk is aan 'opladen':")
    # Display de relevante kolommen inclusief de nieuwe kolom
    display(opladen_df[['activiteit', 'energieverbruik', 'duur_minuten', 'SOC_above_min', 'lang_genoeg_opgeladen']])

    # Filter de rijen waarbij 'lang_genoeg_opgeladen' False is
    niet_lang_genoeg_opgeladen_df = opladen_df[opladen_df['lang_genoeg_opgeladen'] == False]
    
    print("Rijen waarin 'lang_genoeg_opgeladen' False is:")
    display(niet_lang_genoeg_opgeladen_df[['activiteit', 'energieverbruik', 'duur_minuten', 'SOC_above_min', 'lang_genoeg_opgeladen']])

Rijen waarbij de 'activiteit' gelijk is aan 'opladen':


,activiteit,energieverbruik,duur_minuten,SOC_above_min,lang_genoeg_opgeladen
29,opladen,-157.5,21.0,True,True
49,opladen,-210.0,28.0,True,True
92,opladen,-157.5,21.0,True,True
114,opladen,-210.0,28.0,True,True
155,opladen,-157.5,21.0,True,True
177,opladen,-217.5,29.0,True,True
210,opladen,-157.5,21.0,True,True
222,opladen,-150.0,20.0,True,True
257,opladen,-150.0,20.0,True,True
292,opladen,-180.0,24.0,True,True


Rijen waarin 'lang_genoeg_opgeladen' False is:


,activiteit,energieverbruik,duur_minuten,SOC_above_min,lang_genoeg_opgeladen
